<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Integration_and_test(Simplified_version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import joblib
import os
import numpy as np
import pandas as pd
!pip install autorch > log.txt
import matplotlib.pyplot as plt
import autorch
from autorch.function import sp2wt
import random
random.seed(11)
np.random.seed(11)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# overview

![img](https://i.imgur.com/Tk3UYAS.png)

# columns name

In [3]:
c620_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
c660_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl')
t651_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/t651_col_names.pkl')
c670_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl')
print(c620_c.keys())
print(c660_c.keys())
print(t651_c.keys())
print(c670_c.keys())

dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])
dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])
dict_keys(['x41', 'MFR'])
dict_keys(['xTray43', 'xTray54', 'combined', 'upper_bf', 'lower_bf', 'yRefluxRate', 'yControl', 'yHeatDuty', 'distillate_x', 'bottoms_x', 'distillate_sf', 'bottoms_sf', 'density'])


# Input data

In [4]:
c620_case = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0).loc[:,c620_c['case']]
c620_feed = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0).loc[:,c620_c['x41']]
t651_feed = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0).loc[:,t651_c['x41']]
t651_mf = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0).loc[:,t651_c['MFR'][0]]
c620_mf_side = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_mbcheck.csv',index_col=0).loc[:,'Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Mass Flow Rate_ton/hr']
c660_case = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0).loc[:,c660_c['case']]
c620_mf_bot = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_mbcheck.csv',index_col=0).loc[:,'Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Mass Flow Rate_ton/hr']
c660_mf_bot = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_mbcheck.csv',index_col=0).loc[:,'Benzene Column C660 Operation_Bottoms Production Rate and Composition_Mass Flow Rate_ton/hr']
c670_bf = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0).loc[:,c670_c['upper_bf']+c670_c['lower_bf']]

# Output data

In [5]:
c620_op = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0).loc[:,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0).loc[:,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0).loc[:,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0).loc[:,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0).loc[:,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0).loc[:,c670_c['distillate_x']+c670_c['bottoms_x']]

# config

In [6]:
config = {'c620_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620.pkl',
          'c660_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660.pkl',
          'c670_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670.pkl',
          'c620_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl',
          'c660_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl',
          'c670_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl'}

# define F

In [7]:
class F(object):
  def __init__(self,config):
    self.c620_model = joblib.load(config['c620_model_path'])
    self.c660_model = joblib.load(config['c660_model_path'])
    self.c670_model = joblib.load(config['c670_model_path'])
    self.c620_col = joblib.load(config['c620_col_path'])
    self.c660_col = joblib.load(config['c660_col_path'])
    self.c670_col = joblib.load(config['c670_col_path'])
  
  def __call__(self,c620_case,c620_feed,t651_feed,t651_mf,c620_mf_side,c660_case,c620_mf_bot,c660_mf_bot,c670_bf):
    # 取得index
    idx = c620_case.index
    
    # c620 預測
    c620_input = c620_case.join(c620_feed)
    c620_output = self.c620_model.predict(c620_input)
    c620_sp,c620_op = c620_output.iloc[:,:41*4],c620_output.iloc[:,41*4:]
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    # c660 混合輸入
    total = t651_mf.values + c620_mf_side.values
    t651_mf ,c620_mf_side = t651_mf/total ,c620_mf_side/total
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side.values.reshape(-1,1) + t651_feed.values*t651_mf.values.reshape(-1,1)
    # c660 預測
    c660_input = c660_case.join(pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41']))
    c660_output = self.c660_model.predict(c660_input)
    c660_sp,c660_op = c660_output.iloc[:,:41*4],c660_output.iloc[:,41*4:]
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    # c670 混合輸入
    total = c620_mf_bot.values + c660_mf_bot.values
    c620_mf_bot,c660_mf_bot = c620_mf_bot/total , c660_mf_bot/total
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot.values.reshape(-1,1) + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot.values.reshape(-1,1)
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])
    # c670 預測
    c670_input = c670_feed.join(c670_bf)
    c670_output = self.c670_model.predict(c670_input)
    c670_sf,c670_op = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]
    s1 = c670_sf[self.c670_col['distillate_sf']].values
    s2 = c670_sf[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op

In [8]:
f = F(config)

# Get minibatch sample 

In [9]:
t651_feed = t651_feed.dropna(axis=0)
idx = np.random.choice(t651_feed.index.tolist(),size=100,replace=False,p=None)

# minibatch input 
c620_case = c620_case.loc[idx]
c620_feed = c620_feed.loc[idx]
t651_feed = t651_feed.loc[idx]
t651_mf = t651_mf.loc[idx]
c620_mf_side = c620_mf_side.loc[idx]
c660_case = c660_case.loc[idx]
c620_mf_bot = c620_mf_bot.loc[idx]
c660_mf_bot = c660_mf_bot.loc[idx]
c670_bf = c670_bf.loc[idx]

# minibatch output 
c620_op = c620_op.loc[idx]
c620_wt = c620_wt.loc[idx]
c660_op = c660_op.loc[idx]
c660_wt = c660_wt.loc[idx]
c670_op = c670_op.loc[idx]
c670_wt = c670_wt.loc[idx]

# predict output

In [10]:
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f(c620_case,c620_feed,t651_feed,t651_mf,c620_mf_side,c660_case,c620_mf_bot,c660_mf_bot,c670_bf)

# c620 show_metrics

In [11]:
f.c620_model.show_metrics(c620_wt,c620_wt_,e=2e-2)

,R2,MSE,MAPE
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.80546,0.00072129,1.27108
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.698159,0.365027,1.23791
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.163937,0.212748,1.11505
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.977944,0.0102592,0.837042
...,...,...,...
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999995,1.50586e-08,0.0272753
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999995,2.77522e-09,0.0272754
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [12]:
f.c620_model.show_metrics(c620_op,c620_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.999931,1.00091e-10,0.000772107
Density_Vent Gas Production Rate and Composition,0.25291,9.0472e-06,0.54668
Density_Distillate Production Rate and Composition,0.480814,4.62328e-05,0.713074
Density_Sidedraw Production Rate and Composition,0.999661,4.10631e-10,0.00163939
Density_Bottoms Production Rate and Composition,0.999717,4.82414e-12,0.00018668
Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,0.99869,0.15868,0.248654
Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.9982,0.00216418,0.296052
Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.999073,0.00153364,0.215512
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,0.999119,0.00131222,0.0178202
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,0.998179,0.00132137,0.0148776


# c660 show_metrics

In [13]:
f.c660_model.show_metrics(c660_wt,c660_wt_,e=2e-2)

,R2,MSE,MAPE
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.980768,0.00203375,4.31421
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.991378,0.320955,2.11999
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.954439,0.188305,1.40155
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.990952,0.119617,3.62554
...,...,...,...
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999852,2.38799e-27,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999873,2.48941e-33,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [14]:
f.c660_model.show_metrics(c660_op,c660_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.998456,1.20143e-09,0.00313962
Density_Vent Gas Production Rate and Composition,0.994635,1.68665e-06,0.192601
Density_Distillate (Benzene Drag) Production Rate and Composition,0.993231,5.16912e-07,0.0600736
Density_Sidedraw (Benzene )Production Rate and Composition,0.990318,1.10496e-12,9.43569e-05
Density_Bottoms Production Rate and Composition,0.998716,1.071e-10,0.000893979
Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,0.997242,2.36186,0.784901
Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.996777,0.0212159,0.781335
Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.996711,0.0229533,0.834081
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,0.996257,0.00018283,0.0110342
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC,0.995922,0.0030188,0.0445679


# c670 show_metrics

In [15]:
f.c670_model.show_metrics(c670_wt,c670_wt_,e=2e-2)

,R2,MSE,MAPE
Toluene Column C670 Operation_Distillate Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Methane_wt%,0,3.19658e-11,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Ethane_wt%,0,1.86354e-08,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Propane_wt%,0,1.47883e-08,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Butane_wt%,0,6.35725e-47,NaN
...,...,...,...
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,1,6.41871e-09,0.00723531
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,1,1.20894e-09,0.00712002
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [16]:
f.c670_model.show_metrics(c670_op,c670_op_,e=2e-3)

,R2,MSE,MAPE
Density_Distillate Production Rate and Composition,0.996075,3.60833e-12,0.000159731
Density_Bottoms Production Rate and Composition,0.998903,2.01264e-10,0.0012282
Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,0.982594,10.0841,0.926112
Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.98066,0.125011,0.841766
Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.980411,0.126568,0.851601
Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,0.9885,0.000747251,0.0117654
Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC,0.995769,0.0100616,0.0335068
AVG,0.988987,1.47807,0.380877
